### Импортируем и преобразуем данные

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn import datasets

In [2]:
iris = datasets.load_iris(as_frame = True).frame

In [3]:
# Так как данные хорошо известны (3 класса по 50 строк), убираем первые 50 строк принадлежащие к классу Setosa
iris = iris.loc[50:]
iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [4]:
y = iris['target'] -1
X = iris.drop(['target'], axis =1)

### Реализовать самостоятельно логистическую регрессию

In [5]:
def prediction(pred):
        if pred >= 0.5:
            return 1
        else:
            return 0

In [6]:
def my_prediction(params, X):
    pred = params[0] + params[1] * X['sepal length (cm)'] + params[2] * X['sepal width (cm)'] + params[3] * X['petal length (cm)'] + params[4] * X['petal width (cm)']
    return 1 / (1 + np.exp(-(pred)))

### Обучить ее методом градиентного спуска

In [25]:
params = []
params = np.random.normal(size=(5,))
EPOCHS = 200
LEARNING_RATE = 0.1

In [26]:
for _ in range(EPOCHS):
    sigmoid = my_prediction(params, X)
    params[0] -= LEARNING_RATE * np.sum(sigmoid - y) / len(sigmoid)
    number = 1
    for i in X:
        params[number] -= LEARNING_RATE * np.sum((sigmoid - y) * X[i]) / len(sigmoid)
        number += 1
    predictions = my_prediction(params, X).apply(prediction)

In [27]:
pred = my_prediction(params, X)
new_pred = pred.apply(prediction)
1 - np.sum(abs(new_pred - y)) / len(y)

0.95

### Методом nesterov momentum

In [10]:
gamma = 0.9
u_0 = new_u_0 = 0
u = new_u = [0, 0, 0, 0, 0]

In [28]:
for _ in range(EPOCHS):
    sigmoid = my_prediction(params, X)
    new_u_0 = u_0 * gamma + (1 - gamma) * LEARNING_RATE * np.sum(sigmoid - y) / len(sigmoid)
    number = 0
    for i in X:
        new_u[number] = u[number] * gamma + (1 - gamma) * LEARNING_RATE * np.sum((sigmoid - y) * X[i])
        number += 1
    for i in range(len(params)):
        if i == 0:
            params[i] -= new_u_0
        else:
            params[i] -= new_u[i-1]
    u_0 = new_u_0
    u = new_u
    predictions = my_prediction(params, X).apply(prediction)

In [29]:
pred = my_prediction(params, X)
new_pred = pred.apply(prediction)
1 - np.sum(abs(new_pred - y)) / len(y)

0.96

### Методом rmsprop

In [18]:
eps = 0.000001
grad_square = np.zeros(5)

In [30]:
for _ in range(EPOCHS):
    sigmoid = my_prediction(params, X)
    grad_0 = np.sum((sigmoid - y)) / len(sigmoid)
    grad_1 = np.dot((sigmoid - y), X) / len(sigmoid)
    grad = np.hstack((np.array(grad_0), grad_1))
    grad_square = gamma * grad_square + (1 - gamma)  * grad ** 2
    params -= LEARNING_RATE * grad / np.sqrt(grad_square + eps)
    predictions = my_prediction(params, X).apply(prediction)

In [31]:
pred = my_prediction(params, X)
new_pred = pred.apply(prediction)
1 - np.sum(abs(new_pred - y)) / len(y)

0.97